In [1]:
####################################
#ENVIRONMENT SETUP

In [2]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [3]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [4]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [7]:
#IMPORT CLASSES (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [8]:
####################################
#LOADING CLASSES

In [9]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="CalculateMoreVariables", dataName="Moist_Static_Energy",
                                dtype='float32')

=== CM1 Data Summary ===
 Simulation #:   1
 Resolution:     1km
 Time step:      5min
 Vertical levels:34
 Parcels:        1e6
 Data file:      /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_1km_5min_34nz.nc
 Parcel file:    /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_pdata_1km_5min_1e6np.nc
 Time steps:     133

=== DataManager Summary ===
 inputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData
 outputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables
 inputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_5min_34nz/ModelData
 outputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projec

In [10]:
#JOB ARRAY SETUP
UsingJobArray=True

def GetNumJobs(res):
    if res=='1km':
        num_jobs=20
    elif res=='250m': 
        num_jobs=100
    return num_jobs
num_jobs = GetNumJobs(ModelData.res)
SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job

def GetNumElements():
    num_elements = np.arange(ModelData.Ntime)[start_job:end_job]
    return num_elements
num_elements = GetNumElements()


Running timesteps from 0:7 



In [ ]:
####################################
#FUNCTIONS

In [27]:
#Calculation Functions
def GetInputVariables(inputDataDirectory, timeString):    

    ################################# PRESSURE VARIABLES
    p0=1e5
    P = DataManager.GetTimestepData(inputDataDirectory, timeString, VariableName="prs")

   ################################# THERMODYNAMICS
    Rd=287.04;# Rv=461.5
    cpd=1005.7 #+-2.5 #****divide by this
    cpv=1870 #+-25
    cpl=4190 #+-30 #Cpl = Cvl = Cl
    cpi=2106.0 #Cpi = Cvi = Ci
    Lv0=2.501e6
    Lf0=0.3337e6

    def Lv(T): #Kirchoff's formula L_i,ii= L_i,ii0+(Cpii-Cpi)*(T-273.15)
        Llv=Lv0+(cpv-cpl)*(T-273.15)
        return Llv
        
    def Lf(T): #Kirchoff's formula L_i,ii= L_i,ii0+(Cpii-Cpi)*(T-273.15)
        Lli=Lf0+(cpi-cpl)*(T-273.15) 
        return Lli
    
    ################################# TEMPERATURE
    theta = DataManager.GetTimestepData(inputDataDirectory, timeString, VariableName="th")
    T=theta*(P/p0)**(Rd/cpd)

    # print('Specific Humidity'); ################################# Specific Humidity
    rv = DataManager.GetTimestepData(inputDataDirectory, timeString, VariableName="qv")
    qr_data = DataManager.GetTimestepData(inputDataDirectory, timeString, VariableName="qr")
    qc_data = DataManager.GetTimestepData(inputDataDirectory, timeString, VariableName="qc")
    rl=qr_data+qc_data
    rt=rv+rl

    ri=DataManager.GetTimestepData(inputDataDirectory, timeString, VariableName="qi")
    rT=rt+ri

    ################################# Geopotential Height
    Nz,Ny,Nx=ModelData.Nzh,ModelData.Nyh,ModelData.Nxh
    zh_values=ModelData.zh*1000
    Z=np.broadcast_to(zh_values[:, np.newaxis, np.newaxis], (Nz, Ny, Nx))
    g=9.81
    gZ=g*Z

    InputDictionary = {'cpd': cpd, 'cpl': cpl, 'T': T,
                       'Lv': Lv, 'Lf': Lf,
                       'rv': rv, 'ri': ri, 'rt': rt, 'rT': rT,
                       'gZ': gZ
                       }
    return InputDictionary

def VariableCalculation(inputDictionary):
    [cpd,cpl,T] = (inputDictionary[k] for k in ["cpd","cpl","T"])
    [Lv,Lf] = (inputDictionary[k] for k in ["Lv","Lf"])
    [rv,ri,rt,rT] = (inputDictionary[k] for k in ["rv","ri","rt","rT"])
    [gZ] = (inputDictionary[k] for k in ["gZ"])

    ################################# EQUIVALENT POTENTIAL TEMPERATURE
    # MSE=(cpd+rt*cpl)*T + (Lv(T)*rv) + ( (1+rt)*(gZ) ) #no ice
    MSE=(cpd+rT*cpl)*T + (Lv(T)*rv) - (Lf(T)*ri) + ( (1+rT)*(gZ) ) #including ice

    outputDictionary={'MSE': MSE}
    return outputDictionary

In [28]:
####################################
#RUNNING

In [29]:
#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in num_elements:
    if np.mod(t,1)==0: print(f'Current time {t}')

    #getting timestring for loading input data
    timeString = ModelData.timeStrings[t]

    #loading input variables
    inputDictionary = GetInputVariables(DataManager.inputDataDirectory, timeString)

    #calculating variables
    outputDictionary = VariableCalculation(inputDictionary)
    
    #outputting
    DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, timeString, outputDictionary)

Current time 0
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/Moist_Static_Energy/Moist_Static_Energy_1km_5min_34nz_0-00-00.h5
Current time 1
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/Moist_Static_Energy/Moist_Static_Energy_1km_5min_34nz_0-05-00.h5
Current time 2
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/Moist_Static_Energy/Moist_Static_Energy_1km_5min_34nz_0-10-00.h5
Current time 3
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/Moist_Static_Energy/Moist_Static_Energy_1km_5min_34nz

In [ ]:
######################################################

In [30]:
# #READING BACK IN
# def GetVariableData(t, dataName="Moist_Static_Energy"):
#     res = ModelData.res
#     t_res = ModelData.t_res
#     Nz_str = ModelData.Nz_str
#     inputDirectory = os.path.join(DataManager.outputDirectory, f"{res}_{t_res}_{Nz_str}nz", dataName)
#     timeString = ModelData.timeStrings[t]

#     FileName = os.path.join(inputDirectory, f"{dataName}_{res}_{t_res}_{Nz_str}nz_{timeString}.h5")

#     dataDictionary = {}
#     with h5py.File(FileName, 'r') as f:
#         print("Keys in file:", list(f.keys()))
#         for key in f.keys():
#             dataDictionary[key] = f[key][:]
#             print(f"{key}: shape = {dataDictionary[key].shape}, dtype = {dataDictionary[key].dtype}")
#     return dataDictionary

# dataDictionary = GetVariableData(t=0)

Keys in file: ['MSE']
MSE: shape = (34, 200, 512), dtype = float32
